# 最終課題

### 課題内容
『武蔵野大学のWebサイトのサイトマップを抽出する』
・武蔵野大学Webサイトのトップページにアクセス
・同一ドメインの全てのリンク（コメントアウトされていないもの）を辿り，全ページのURLと<title>を辞書型変数に格納する
key：URL
value：<title></title>で挟まれた文字列
・辞書型変数を print() で表示する


・注意
Webサイトにアクセスする際は必ず time.sleep(秒数) を使って負荷軽減をすること
課題用のipynb形式のファイルを作成し，githubフローでファイルの変更履歴を管理すること


In [ ]:
#　インポート
import requests
from bs4 import BeautifulSoup
import time

In [2]:
url = "https://www.musashino-u.ac.jp/"
sitemap = {}    # サイトマップ用の辞書を作成


In [3]:
res = requests.get(url)
time.sleep(1) # アクセス負荷軽減
soup = BeautifulSoup(res.content, 'html.parser')
